## 3.softmax回归

**学习目标**

1. 熟悉softmax回归算法和模型构建方法

2. 熟练使用Softmax激活函数

3. 熟练使用交叉熵（CE）损失函数

4. 能够使用torchvision.datasets.MNIST加载MNIST数据集
****

20世纪50年代，统计学家和数学家开始研究如何将一组实数转换为概率分布。自然而然就想到了将Sigmoid函数从二分类任务推广到多分类任务中，并将其命名为Softmax函数。1980年代，Softmax函数被引入神经网络，成为多分类问题的标准输出层激活函数。1989年，Yann LeCun等人提出了卷积神经网络（CNN），并在手写数字识别任务中使用了Softmax函数。随着深度学习的兴起，Softmax函数成为神经网络中多分类问题的标准输出层激活函数。

### 3.1 softmax回归

1、softmax函数

softmax函数是多分类问题的输出层激活函数，它将输入的特征向量映射到一个概率分布。softmax函数的公式如下：

$$
softmax(x_i) = \frac{e^{x_i}}{\sum_{j=1}^{N} e^{x_j}}
$$

其中，$x_i$是输入向量的第i个元素，$N$是类别的数量。softmax函数将输入的特征向量映射到一个概率分布，其中第i个元素表示该样本属于第i个类别的概率。softmax函数的输出是一个N维的向量，其中第i个元素表示该样本属于第i个类别的概率。

2、softmax回归

softmax回归是多分类问题的一种常用方法。它将输入的特征向量映射到一个概率分布，并通过softmax函数输出每个类别的概率。softmax回归的损失函数通常使用交叉熵损失函数。


### 3.2 交叉熵（Cross-Entropy, CE）损失函数

交叉熵损失函数通常用于多分类问题。其计算公式如下：

$$
\text{CELoss} = -\frac{1}{n} \sum_{i=1}^{n} \sum_{j=1}^{C} y_{ij} \log(p_{ij})
$$

其中：

-  n  是样本的数量。

-  C  是类别的数量。

- $ y_{ij} $ 是样本  i  的真实标签（独热编码形式，即如果样本  i  属于类别  j ，则  y_{ij} = 1 ，否则  y_{ij} = 0 ）。

- $p_{ij}$是模型对样本  i  属于类别  j  的预测概率。

- 当样本  i  的真实标签为类别  j （即  $y_{ij}$ = 1 ）：
  - $ p_{ij} $ 越大，表示预测越接近真实标签，损失越小。
  - 如果 $ p_{ij} $ 接近 1，则 $ \log(p_{ij}) $ 接近 0，损失较小。
  - 如果 $ p_{ij} $ 接近 0，则 $ \log(p_{ij}) $ 接近负无穷，损失较大。

- 当样本  i  的真实标签不是类别  j （即  $y_{ij}$ = 0 ）：
  - 由于 $ y_{ij} $= 0 ，这部分的损失为 0，因此不会对总损失产生影响。

最终的损失是所有样本在所有类别上的损失之和。

交叉熵损失函数的一个潜在问题是，当类别不平衡时，某些类别的样本数量远远少于其他类别，这可能导致模型对少数类别的预测性能较差。在这种情况下，损失函数可能会偏向于多数类别，从而忽略少数类别。为了解决这个问题，可以考虑使用加权交叉熵损失（Weighted Cross-Entropy Loss）或其他处理类别不平衡的方法，如采样技术或使用 Focal Loss 等。

3.3 FashionMNIST数据集

Fashion-MNIST 是 Zalando 文章图像的数据集，包含 60,000 个训练样本和 10,000 个测试样本。每个样本都是一张 28x28 的灰度图像，并附有一个从 10 个类别中提取的标签。Zalando 将 Fashion-MNIST 作为MNIST数据集的替代品，用于基准测试机器学习算法。它与MNIST数据集具有相同的图像尺寸和训练/测试集划分结构。

<img src="./images/dataset-cover.png" style="zoom:40%;" />

每张图像的高度和宽度均为 28 像素，总共 784 个像素。每个像素都有一个与之关联的像素值，表示该像素的明暗程度，数值越大表示越暗。该像素值是一个介于 0 到 255 之间的整数。训练和测试数据集各有 785 列。第一列包含类别标签，表示服装的类型。其余列包含与图像相关的像素值。

标签
每个训练和测试样本都被分配到以下标签之一：

0 T恤/上衣
1 裤子
2 套头衫
3 裙子
4 外套
5 凉鞋
6 衬衫
7 运动鞋
8 包
9 短靴

0 T-shirt/top
1 Trouser
2 Pullover
3 Dress
4 Coat
5 Sandal
6 Shirt
7 Sneaker
8 Bag
9 Ankle boot

***

### 3.4 softmax回归的应用——时尚服饰识别

1.导入必需的模块

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

2.数据集的加载和预处理

In [2]:
# 定义转换操作
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transforms.Compose 将transforms.ToTensor和transforms.Normalize等转换步骤组合起来，创建的预处理流程可以一次性应用到图像上。这使得数据预处理步骤的代码更加简洁和易于管理。

 在transforms.Normalize((0.5,), (0.5,)) 中：

第一个参数 (0.5,) 表示均值，它是一个单元素的元组，意味着所有通道的均值都是 0.5。由于只有一个元素，这通常用于灰度图像。

第二个参数 (0.5,) 表示标准差，它也是一个单元素的元组，意味着所有通道的标准差都是 0.5。

当你将这个变换应用到图像数据上时，每个通道的每个像素值将被减去均值 0.5 然后除以标准差 0.5。

这将导致变换后的像素值范围从原始的 [0, 1] 或 [0, 255] 变为 [-1, 1]。这种标准化有助于模型更快地收敛，因为它确保了不同通道和不同像素值的分布更加一致。

实际使用中，均值和标准差通常是根据整个数据集的统计数据计算得到的，以便更好地反映数据的分布特性。

In [3]:
# 下载训练集和测试集
train_dataset = datasets.FashionMNIST(root='./datasets', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./datasets', train=False, download=True, transform=transform)

datasets.MNIST 是 PyTorch 的 torchvision.datasets 模块中的一个类，它用于加载和处理 MNIST 数据集。

使用 datasets.MNIST 加载数据集时，你可以指定数据集的根目录、转换（transforms）、下载方式等参数。这个类提供了一个方法 __getitem__ 来访问数据集中的每个样本，以及 __len__ 方法来获取数据集的大小。

使用 datasets.MNIST 类加载了训练集和测试集，通过 download=True 参数自动下载数据集（如果本地没有的话）。

In [4]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

DataLoader的作用是将数据集封装用于后面的训练，我们使用DataLoader来加载训练集和验证集。

3.模型构建

（1）参数设置

In [5]:
input_dim = 28*28  # MNIST图像大小为28x28
num_classes = 10   # 总共有10个类别（0到9）
num_epochs = 10

（2）构建Softmax回归模型

In [6]:
model = nn.Sequential(nn.Linear(input_dim, num_classes))

（3）定义损失函数和优化器

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

nn.CrossEntropyLoss()在PyTorch中已经内置了softmax函数。这个损失函数通常用于多分类问题，它结合了softmax层和负对数似然损失（negative log likelihood loss），因此你不需要在损失函数之前手动添加softmax层。
nn.CrossEntropyLoss的输入要求是未归一化的分数（即模型的原始输出），它将这些分数通过softmax函数转换为概率分布，然后计算这些概率与目标类别之间的交叉熵损失。

Softmax函数是一种常用的激活函数，特别是在处理多分类问题时。它将一个向量或一组实数转换成概率分布，使得每个元素的值都在0到1之间，并且所有元素的和为1。这使得Softmax函数非常适合用作神经网络输出层的激活函数，用于预测多个类别的概率。

在多分类问题中，Softmax函数通常与交叉熵损失（Cross-Entropy Loss）一起使用，这有助于优化模型以正确分类数据。

4.模型训练

In [8]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0  # 初始化总损失累加器
    
    for data, labels in train_loader:
        
        outputs = model(data.view(-1, input_dim))
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()  # 累加损失值

    # 计算平均损失并打印
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.5f}')

Epoch [1/10], Loss: 0.68753
Epoch [2/10], Loss: 0.51900
Epoch [3/10], Loss: 0.48629
Epoch [4/10], Loss: 0.46839
Epoch [5/10], Loss: 0.45670
Epoch [6/10], Loss: 0.44831
Epoch [7/10], Loss: 0.44179
Epoch [8/10], Loss: 0.43641
Epoch [9/10], Loss: 0.43177
Epoch [10/10], Loss: 0.42775


5.模型测试

In [9]:
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    
    for data, labels in test_loader:
        
        outputs = model(data.view(-1, input_dim))
        # print(outputs[0])
        _, predicted = torch.max(outputs, 1)  # _ 最大值，predicted 最大值索引
        # print(predicted[0], labels[0])
        # print(labels.shape, labels.size(0))
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total}%')

Accuracy: 83.64%
